In [1]:
import tensorflow as tf

In [2]:
import numpy as np

In [4]:
y_pred_3d_mask = tf.Variable(y_pred)
y_pred = tf.Variable([1, 20, 2, 40, 3, 60])
y_train = tf.Variable([10, 200, 20, 400, 30, 600])

In [6]:
tf.shape(y_pred)

<tf.Tensor 'Shape:0' shape=(1,) dtype=int32>

# Simple implementation

In [22]:
from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin xmax ymin ymax zmin zmax')

ra = Rectangle(10, 30, 10, 30, 10, 30)
rb = Rectangle(20, 40, 20, 40, 20, 40)
# intersection here is (3, 3, 4, 3.5), or an area of 1*.5=.5

def intersection_volume(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    dz = min(a.zmax, b.zmax) - max(a.zmin, b.zmin)
    if (dx>=0) and (dy>=0) and (dz>=0):
        return dx*dy*dz

def total_volume(a, b):
    vol_a = (a.xmax - a.xmin) * (a.ymax - a.ymin) * (a.zmax - a.zmin)
    vol_b = (b.xmax - b.xmin) * (b.ymax - b.ymin) * (b.zmax - b.zmin)
    
    union = vol_a + vol_b - intersection_volume(a,b)
    return union
    
print area(ra, rb)
print(total_volume(ra, rb))

1000
15000


# Implement in numpy and view in Mayavi to ensure

In [34]:
import numpy as np
import mayavi.mlab as mlab
import sys
sys.path.append('..')

from modules.mischelpers import createDense

In [35]:
seg = np.zeros((240, 240, 155))
ra = np.array([10, 30, 10, 30, 10, 30])
rb = np.array([20, 40, 20, 40, 20, 40])


dense_bbox_orig = createDense(ra, seg)
dense_bbox_pred = createDense(rb, seg)

# start the visualization process
mfig = mlab.figure() # size=(1024, 1024)

mlab.contour3d(dense_bbox_orig, opacity=0.5, color=(0,1,0), transparent=True)
mlab.contour3d(dense_bbox_pred, opacity=0.5, color=(1,0,1), transparent=True)

mlab.show()

# Implement in tensorflow

In [8]:
def tf_count(t, val):
    elements_equal_to_value = tf.equal(t, val)
    as_ints = tf.cast(elements_equal_to_value, tf.int32)
    count = tf.reduce_sum(as_ints)
    return count

def bbox_iou_masks(y_true, y_pred):
    # split the bounding boxes into individual tensors
    xmin_true, xmax_true, ymin_true, ymax_true, zmin_true, zmax_true = tf.split(y_true, 6, axis=1)
    xmin_pred, xmax_pred, ymin_pred, ymax_pred, zmin_pred, zmax_pred = tf.split(y_pred, 6, axis=1)
    
    y_true_tmp = tf.ones([xmax_true, ymax_true, zmax_true])
    y_pred_tmp = tf.ones([xmax_pred, ymax_pred, zmax_pred])
    
    y_true_blacks = tf.zeros([240, 240, 155], dtype=tf.float32, name=None)
    y_pred_blacks = tf.zeros([240, 240, 155], dtype=tf.float32, name=None)
    
    y_true_mask = y_true_blacks[xmin_true:xmax_true, ymin_true:ymax_true, zmin_true:zmax_true].assign(y_true_tmp)
    y_pred_mask = y_pred_blacks[xmin_pred:xmax_pred, ymin_pred:ymax_pred, zmin_pred:zmax_pred].assign(y_pred_tmp)
    
    # count all non-zeros in the masks to get the union value
    
    total_mask = y_true_mask + y_pred_mask
    
    union = tf_count(total_mask, 1.0)
    intersection = tf_count(total_mask, 2.0)
    
    iou = intersection / (union + 0.0001)
    
    return intersection, union, iou

In [3]:
import tensorflow as tf
import keras.backend as K
import numpy as np
from keras.losses import mean_absolute_error
import mayavi.mlab as mlab
import sys
sys.path.append('..')
from modules.mischelpers import createDense


def bbox_overlap_iou(y_true, y_pred):
    # split the bounding boxes into individual tensors
    xmin_true, xmax_true, ymin_true, ymax_true, zmin_true, zmax_true = tf.split(y_true, 6, axis=1)
    xmin_pred, xmax_pred, ymin_pred, ymax_pred, zmin_pred, zmax_pred = tf.split(y_pred, 6, axis=1)

    dx = K.minimum(xmax_true, xmax_pred) - K.maximum(xmin_true, xmin_pred)
    dy = K.minimum(ymax_true, ymax_pred) - K.maximum(ymin_true, ymin_pred)
    dz = K.minimum(zmax_true, zmax_pred) - K.maximum(zmin_true, zmin_pred)

    # find the intersection volume of the boxes
    intersection = dx * dy * dz
    
    # find the total volume and then find the union
    vol_true = (xmax_true - xmin_true) * (ymax_true - ymin_true) * (ymax_true - ymin_true)
    vol_pred = (xmax_pred - xmin_pred) * (ymax_pred - ymin_pred) * (ymax_pred - ymin_pred)

    # find the union now
    union = vol_true + vol_pred - intersection
    
    iou = intersection / (union + 0.0001)
    
    return intersection, union, iou

def iou_with_mse(y_true, y_pred):
    # split the bounding boxes into individual tensors
    xmin_true, xmax_true, ymin_true, ymax_true, zmin_true, zmax_true = tf.split(y_true, 6, axis=1)
    xmin_pred, xmax_pred, ymin_pred, ymax_pred, zmin_pred, zmax_pred = tf.split(y_pred, 6, axis=1)

    dx = K.minimum(xmax_true, xmax_pred) - K.maximum(xmin_true, xmin_pred)
    dy = K.minimum(ymax_true, ymax_pred) - K.maximum(ymin_true, ymin_pred)
    dz = K.minimum(zmax_true, zmax_pred) - K.maximum(zmin_true, zmin_pred)

    intersection = dx * dy * dz
    intersection = K.relu(intersection)

    # find the total volume and then find the union
    vol_true = tf.abs((xmax_true - xmin_true) * (ymax_true - ymin_true) * (ymax_true - ymin_true))
    vol_pred = tf.abs((xmax_pred - xmin_pred) * (ymax_pred - ymin_pred) * (ymax_pred - ymin_pred))

    # find the union now
    union = vol_true + vol_pred - intersection

    iou = intersection / union

    return -1000*iou, mean_absolute_error(y_true, y_pred), intersection, union, iou

In [9]:
bboxes1 = tf.placeholder(tf.float32)
bboxes2 = tf.placeholder(tf.float32)

# overlap_op = iou_with_mse(bboxes1, bboxes2)

overlap_op = bbox_iou_masks(bboxes1, bboxes2)

bboxes1_vals = np.array([10, 30, 10, 30, 10, 30])
bboxes2_vals = np.array([20, 40, 20, 40, 20, 40])

with tf.Session() as sess:
    # take care of the data shape that this expects. Notice the [] around. 
    overlap = sess.run(overlap_op, feed_dict={
        bboxes1: np.array([bboxes1_vals]),
        bboxes2: np.array([bboxes2_vals]),
    })

    print('Weighted-IOU {}'.format(overlap[0]))
    print('Unweighted-IOU {}'.format(overlap[-1]))
    print('MAE {}'.format(overlap[1]))
    print('Intersection {}'.format(overlap[2]))
    print('Union {}'.format(overlap[3]))
    
seg = np.zeros((240, 240, 155))
# ra = np.array([10, 30, 10, 30, 10, 30])
# rb = np.array([20, 40, 20, 40, 20, 40])


dense_bbox_orig = createDense(np.array(bboxes1_vals), seg)
dense_bbox_pred = createDense(np.array(bboxes2_vals), seg)

# start the visualization process
mfig = mlab.figure() # size=(1024, 1024)

mlab.contour3d(dense_bbox_orig, opacity=0.5, color=(0,1,0), transparent=True)
mlab.axes()
mlab.contour3d(dense_bbox_pred, opacity=0.1, color=(1,0,1), transparent=True)
mlab.axes()
mlab.show()

TypeError: Expected int32, got list containing Tensors of type '_Message' instead.

In [40]:
bboxes1_vals[0]

[10, 30, 10, 30, 10, 30]

In [37]:
seg[bboxes1_vals[0]:bboxes1_vals[1]]

IndexError: list index out of range